# Grovers Algorithm

In [1]:
my_list = [1, 3, 5, 2, 4, 8, 5, 8, 0, 7, 6]

# How to find 7? 

# We use Oracle to find it. Oracle is black box to ask whether it is 7 or not

def the_oracle(my_input):
    winner = 7
    if my_input is winner:
        response = True
    else:
        response = False
    return response

In [4]:
for index, trial_number in enumerate(my_list):
    if the_oracle(trial_number) is True:
        print('Winner found at index %i'%index)
        print('%i calls to the Oracle used' %(index+1))
        break

Winner found at index 9
10 calls to the Oracle used


### Results

In the classical case, we need to search for every single number in my_list, which means that it will take O(N) as a time complexity. 

Instead, in Quantum Computing, this will change it to O($\sqrt(N)$), which is more efficient than classical computing.mm 

### Oracle
Oracle in Quantum Computing is simply flip over the sign if it's the winner.  
This can be done by Controlled Z gate. 

We also need one more ingredient which is called **Amplitude Amplification** or specifically Reflection. 

By combining Oracle and Reflection which we called **Grover's Diffusion Operator**


In [5]:
from qiskit import *
import matplotlib.pyplot as plt
import numpy as np

In [6]:
# Define the oracle circuit
oracle = QuantumCircuit(2, name = 'oracle')
oracle.cz(0, 1)
oracle.to_gate()
oracle.draw()

q_0: ─■─
      │ 
q_1: ─■─

In [7]:
# To verify that oracle doing right things we call Aer 
backend = Aer.get_backend('statevector_simulator')
grover_circ = QuantumCircuit(2,2)
grover_circ.h([0,1]) # Hadamard to qubit 0 and 1 
grover_circ.append(oracle, [0,1])
grover_circ.draw()

┌───┐┌─────────┐
q_0: ┤ H ├┤0        ├
     ├───┤│  oracle │
q_1: ┤ H ├┤1        ├
     └───┘└─────────┘
c: 2/════════════════

In [9]:
job = execute(grover_circ, backend)
result = job.result()

state_vector = result.get_statevector()
np.around(state_vector, 2)

#This is where we made the orthogonal states from the winner vector. 

array([ 0.5+0.j,  0.5+0.j,  0.5+0.j, -0.5+0.j])

From this results, we need to find the probability of getting for the highest which is the winner by squaring the state factor by using hermitian conjugate. 

In [11]:
# Reflection Operator (Amplifying the probability of the winning state and reducing the non-winning state probability
# Reflection Operator define as this |s><s| - identity matrix. 
# This helps to bring it back to all positive vector which helps to closer to the winner vector. 

reflection = QuantumCircuit(2, name = "reflection")
reflection.h([0, 1])
reflection.z([0, 1])
reflection.cz(0,1) # this helps to transform into negative states
reflection.h([0, 1]) # bring it back to positive 
reflection.to_gate()

reflection.draw()

┌───┐┌───┐   ┌───┐
q_0: ┤ H ├┤ Z ├─■─┤ H ├
     ├───┤├───┤ │ ├───┤
q_1: ┤ H ├┤ Z ├─■─┤ H ├
     └───┘└───┘   └───┘

In [13]:
backend = Aer.get_backend("qasm_simulator")
grover_circ = QuantumCircuit(2,2)
grover_circ.h([0,1])
grover_circ.append(oracle, [0,1])
grover_circ.append(reflection, [0,1])
grover_circ.measure([0,1], [0,1])

grover_circ.draw()

┌───┐┌─────────┐┌─────────────┐┌─┐   
q_0: ┤ H ├┤0        ├┤0            ├┤M├───
     ├───┤│  oracle ││  reflection │└╥┘┌─┐
q_1: ┤ H ├┤1        ├┤1            ├─╫─┤M├
     └───┘└─────────┘└─────────────┘ ║ └╥┘
c: 2/════════════════════════════════╩══╩═
                                     0  1

The above circuit shows that we first prepared superposition state by two Hadamard Gates. Then, add oracle operator and reflection operator to bring it back to positive states. Then after measure we should get 11 state

In [ ]:
job = e